# Extração Sentenças e Classes

In [1]:
from nltk.tokenize import word_tokenize, sent_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords
from tqdm import tqdm
from collections import defaultdict
from ast import literal_eval
import json

In [2]:
#Diretorios dos datasets
filenames = ['cameras.xml','cells.xml','dvds.xml','laptops.xml','routers.xml']

# files gabaritos
gabaritos = ['DadosCamera.txt', 'DadosCells.txt', 'DadosDvds.txt', 'DadosLaptops.txt', 'DadosRouters.txt']

In [3]:
#Diretorios dos arquivos das sentencas
save_sentencas = ['sentencas-camera.txt','sentencas-cells.txt','sentencas-dvds.txt','sentencas-laptops.txt','sentencas-routers.txt']

#Diretorios dos arquivos de palavras-sentencas
save_palavras = ['palavras-camera.json','palavras-cells.json','palavras-dvds.json','palavras-laptops.json','palavras-routers.json']

#Diretorios dos arquivos de frequencia
save_frequencias = ['frequencia-camera.json','frequencia-cells.json','frequencia-dvds.json','frequencia-laptops.json','frequencia-routers.json']

In [4]:
#Função que retorna a sentenca
def get_Sentence(s):
    resp = ""
    for i in range(len(s)):
        if(s[i] == '>'):
            resp = s[i+1:]
            break
    resp = resp.strip()
    return resp

def index_in_list(a_list, index):
    return len(a_list) > index

In [5]:
def get_class_products(filename):
    arq = open('../../datasets/gabaritos/' + filename, 'r')
    cla = []
    for line in arq.readlines():
        idx = line.index(':')
        cla.append(line[:idx])
    arq.close()
    
    return cla

In [6]:
products = ['cameras', 'cells', 'dvds', 'laptops', 'routers']
uknow = {}
for value in products:
    uknow[value] = list()

In [7]:
for i, filename in enumerate(filenames):
    documents = [] # variável que guarda os documentos do dominio
    produto = filename.replace('.xml', '')
    class_product = get_class_products(gabaritos[i])
    dataset_product = {}
    for cla in class_product:
        dataset_product[cla] = defaultdict(list)
    
    sentencas = [] #Variavel que guarda as sentencas do documento
    sentencas_not_class = {} #Variavel que guarda os documentos do dominio
    lista_index = []
    
    arq = open('../../datasets/' + filename,'r')
    for j, linha in enumerate(arq.readlines()):
        if('<sentence' in linha):
            s = linha.replace('</sentence>','')
            s = s.split('idSentence=')[1]
            s = get_Sentence(s) #Obtendo a sentenca
            sentencas.append(s)
            sentencas_not_class[len(sentencas) - 1] = s
            
        if('<opinion>' in linha):
            
            p = linha.replace('/opinion','opinion')
            p = p.replace('<opinion>','')
            p = p.split('"')
            aspect_class = p[7].lower()
            palavra = p[1].lower() #Obtendo o opinion (atributo)
            tipo = p[-2]
            if(tipo != 'anaphora' and (palavra not in set(stopwords.words('english')))):
                if(aspect_class not in dataset_product.keys()):
                    dataset_product[aspect_class] = defaultdict(list)
                    
                dataset_product[aspect_class][palavra].append(sentencas[len(sentencas) - 1])
                lista_index.append(len(sentencas) - 1)
                
        if('</review>' in linha):
            lista_set = set(lista_index)
            documents.append(sentencas)
            
            for key in sentencas_not_class.keys():
                if key in lista_set:
                    continue
                else:
                    uknow[produto].append(sentencas[key])     
            
            lista_index = []
            sentencas = []
            sentencas_not_class = {}
                
    arq.close()
    
    #Salvando as palavras com suas sentencas nos arquivos
    data_words = {}
    with open('../../datasets_processed/' + save_palavras[i],'w') as fp:
        for key in tqdm(dataset_product.keys(), desc='Palavras e Setencas'):
            for value in sorted(dataset_product[key].keys()):
                data_words[value] = dataset_product[key][value]
        
        json.dump(data_words, fp)
    
    
    #salvando sentenças em arquivos
    arq = open('../../datasets_processed/sentencas/' + save_sentencas[i],'w')
    for doc in documents:
        for sent in doc:
            arq.write(sent + '\n')
        arq.write('\n')
    arq.close()
    
    # lendo sentencas
    arq = open('../../datasets_processed/sentencas/' + save_sentencas[i],'r')
    texto = arq.read().lower() #Obtem todos os documentos
    arq.close()
    
    # salvando frequecia de palavras
    data_freq = {}
    with open('../../datasets_processed/frequencias/' + save_frequencias[i],'w') as fp:
        for key in tqdm(dataset_product.keys(), desc='Frequencia Palavra'):
            for value in sorted(dataset_product[key].keys()):
                frequencia = 0
                frequencia = texto.count(value)
                data_freq[value] = frequencia
        
        json.dump(data_freq, fp)
    

Frequencia Palavra: 100%|██████████| 20/20 [00:00<00:00, 6684.68it/s]


In [8]:
dataset_product.keys()

dict_keys(['Others', 'Security', 'Price', 'General', 'Dimension', 'Port', 'Accessory', 'Software', 'Speed', 'CoverageArea', 'others', 'security', 'price', 'general', 'dimension', 'port', 'accessory', 'software', 'speed', 'coveragearea'])